**Objective : To perform RAG-SQL on Bank Customers dataset**

### Load all Essential Libraries

In [1]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

### Initialize the LLM

In [2]:
OPENAI_API_KEY = "ENTER_OPENAI_API_KEY"
llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

C:\Users\gkris\AppData\Local\Temp\ipykernel_22848\1703113971.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)


The API key is provided when you sign up for OpenAI's services and is required to access their language models (like GPT-3 or GPT-4). This key will be used later to interact with OpenAI’s language models through the ChatOpenAI class."temperature=0" This controls the "creativity" or randomness of the language model

### Database Setup

In [3]:
host = 'localhost'
port = '3306'
username = 'root'
#password = ''
database_schema = 'assignmentsql'
mysql_uri = f"mysql+pymysql://{username}@{host}:{port}/{database_schema}"
#mysql_uri = f"mysql+pymysql://{username:password}@{host}:{port}/{database_schema}"
db = SQLDatabase.from_uri(mysql_uri, include_tables=['book_'],sample_rows_in_table_info=2)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

This creates an instance of SQLDatabase, which establishes a connection to the MySQL database using the previously constructed mysql_uri. "mysql_uri" is the URI containing the credentials and details to connect to the MySQL database.db_chain" creates a SQLDatabaseChain, which is responsible for converting natural language queries into SQL queries, running them against the MySQL database, and retrieving results. The chain uses both the language model (llm) and the database connection (db).

### Function to Retrieve and Clean Database Query Result

In [4]:
def retrieve_from_db(query: str) -> str:
    db_context = db_chain(query)
    db_context = db_context['result'].strip()
    return db_context

The retrieve_from_db function processes a query by passing it through a db_chain. It extracts the 'result' from the chain's output and removes any leading/trailing whitespace using .strip(). Finally, it returns the cleaned result as a string.

### Function to Generate Loan Scheme Information Response Based on User Query

In [5]:
def generate(query: str) -> str:
    db_context = retrieve_from_db(query)
    
    system_message = """You are a bank loan expert specializing in loan schemes.
        Your task is to answer users' questions by providing relevant information from a database of loan applicants.
        The database contains the following information for each applicant:
        - Income of the person
        - Age of the person
        - Experience in their field of work
        - Marital status
        - House ownership status
        - Car ownership status
        - Profession of the person
        - City of residence
        - State of residence
        - Current job status
        - Current housing status
        - Risk flag (indicating the likelihood of loan repayment issues)
    
        """
    
    human_qry_template = HumanMessagePromptTemplate.from_template(
        """Input:
        {human_input}
        
        Context:
        {db_context}
        
        Output:
        """
    )
    messages = [
      SystemMessage(content=system_message),
      human_qry_template.format(human_input=query, db_context=db_context)
    ]
    response = llm(messages).content
    return response

The generate function first retrieves the relevant database context using the retrieve_from_db function. It then formats a prompt for a language model with a system message that provides the context of loan applicants' data, followed by the user's query. The language model is then invoked to generate and return a response based on the input query and context.

### Generate and return the response from LLM

In [6]:
generate("how many applicants are there ")

C:\Users\gkris\AppData\Local\Temp\ipykernel_22848\3606437300.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  db_context = db_chain(query)




> Entering new SQLDatabaseChain chain...
how many applicants are there 
SQLQuery:SELECT COUNT(`Id`) AS `Total_Applicants` FROM book_
SQLResult: [(2499,)]
Answer:There are 2499 applicants.
> Finished chain.


C:\Users\gkris\AppData\Local\Temp\ipykernel_22848\3520852297.py:36: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(messages).content


'There are 2499 applicants in the database.'

In [7]:
generate("provide me names of states of applicants in the table")



> Entering new SQLDatabaseChain chain...
provide me names of states of applicants in the table
SQLQuery:SELECT DISTINCT `STATE` FROM book_
SQLResult: [('Madhya_Pradesh',), ('Maharashtra',), ('Kerala',), ('Odisha',), ('Tamil_Nadu',), ('Gujarat',), ('Rajasthan',), ('Telangana',), ('Bihar',), ('Andhra_Pradesh',), ('West_Bengal',), ('Haryana',), ('Puducherry',), ('Karnataka',), ('Uttar_Pradesh',), ('Himachal_Pradesh',), ('Punjab',), ('Tripura',), ('Uttarakhand',), ('Jharkhand',), ('Mizoram',), ('Assam',), ('Jammu_and_Kashmir',), ('Delhi',), ('Chhattisgarh',), ('Chandigarh',), ('Uttar_Pradesh[5]',), ('Manipur',), ('Sikkim',)]
Answer:Madhya_Pradesh, Maharashtra, Kerala, Odisha, Tamil_Nadu, Gujarat, Rajasthan, Telangana, Bihar, Andhra_Pradesh, West_Bengal, Haryana, Puducherry, Karnataka, Uttar_Pradesh, Himachal_Pradesh, Punjab, Tripura, Uttarakhand, Jharkhand, Mizoram, Assam, Jammu_and_Kashmir, Delhi, Chhattisgarh, Chandigarh, Uttar_Pradesh[5], Manipur, Sikkim
> Finished chain.


'Madhya_Pradesh, Maharashtra, Kerala, Odisha, Tamil_Nadu, Gujarat, Rajasthan, Telangana, Bihar, Andhra_Pradesh, West_Bengal, Haryana, Puducherry, Karnataka, Uttar_Pradesh, Himachal_Pradesh, Punjab, Tripura, Uttarakhand, Jharkhand, Mizoram, Assam, Jammu_and_Kashmir, Delhi, Chhattisgarh, Chandigarh, Uttar_Pradesh, Manipur, Sikkim'

In [8]:
generate(" provide me applicants who are having car ownership and from states UttarPradesh, Gujarat ")



> Entering new SQLDatabaseChain chain...
 provide me applicants who are having car ownership and from states UttarPradesh, Gujarat 
SQLQuery:SELECT `Id`, `Profession`, `Car_Ownership`, `STATE` 
FROM book_
WHERE `Car_Ownership` = 'yes' AND (`STATE` = 'UttarPradesh' OR `STATE` = 'Gujarat')
LIMIT 5;
SQLResult: [(64, 'Technical_writer', 'yes', 'Gujarat'), (93, 'Firefighter', 'yes', 'Gujarat'), (98, 'Software_Developer', 'yes', 'Gujarat'), (129, 'Chef', 'yes', 'Gujarat'), (352, 'Artist', 'yes', 'Gujarat')]
Answer:Applicants who have car ownership and are from the states of UttarPradesh and Gujarat are as follows:
1. Profession: Technical_writer, Car_Ownership: yes, State: Gujarat
2. Profession: Firefighter, Car_Ownership: yes, State: Gujarat
3. Profession: Software_Developer, Car_Ownership: yes, State: Gujarat
4. Profession: Chef, Car_Ownership: yes, State: Gujarat
5. Profession: Artist, Car_Ownership: yes, State: Gujarat
> Finished chain.


'Here are the applicants who have car ownership and are from the states of UttarPradesh and Gujarat:\n1. Profession: Technical_writer, Car_Ownership: yes, State: Gujarat\n2. Profession: Firefighter, Car_Ownership: yes, State: Gujarat\n3. Profession: Software_Developer, Car_Ownership: yes, State: Gujarat\n4. Profession: Chef, Car_Ownership: yes, State: Gujarat\n5. Profession: Artist, Car_Ownership: yes, State: Gujarat'

In [9]:
generate("How many applicants has martial status ,married")



> Entering new SQLDatabaseChain chain...
How many applicants has martial status ,married
SQLQuery:SELECT COUNT(*) AS Married_Applicants
FROM book_
WHERE Married = 'married'
LIMIT 5;
SQLResult: [(262,)]
Answer:Final answer: There are 262 applicants who have the marital status 'married'.
> Finished chain.


"There are 262 applicants who have the marital status 'married'."

In [10]:
generate("how many members are having red flags and high income")



> Entering new SQLDatabaseChain chain...
how many members are having red flags and high income
SQLQuery:SELECT COUNT(`Id`) AS `HighIncome_RedFlags`
FROM book_
WHERE `Income` > 5000000 AND `Risk_Flag` = 1
SQLResult: [(225,)]
Answer:There are 225 members who have red flags and high income.
> Finished chain.


'There are 225 members who have red flags and high income.'

In [11]:
generate("Provide me how many members are feom state Maharashtra and theri hose status along with current city they are living ")



> Entering new SQLDatabaseChain chain...
Provide me how many members are feom state Maharashtra and theri hose status along with current city they are living 
SQLQuery:SELECT COUNT(*) AS `Number_of_Members`, `House_Ownership`, `CITY`
FROM book_
WHERE `STATE` = 'Maharashtra'
GROUP BY `House_Ownership`, `CITY`
SQLResult: [(1, 'norent_noown', 'Dhule'), (1, 'norent_noown', 'Jalgaon'), (1, 'norent_noown', 'Jalna'), (1, 'norent_noown', 'Satara'), (1, 'norent_noown', 'Thane'), (1, 'owned', 'Ahmednagar'), (1, 'owned', 'Akola'), (1, 'owned', 'Amravati'), (1, 'owned', 'Kalyan-Dombivli'), (2, 'owned', 'Nagpur'), (2, 'owned', 'Nashik'), (1, 'owned', 'Panvel'), (1, 'owned', 'Parbhani'), (2, 'owned', 'Pimpri-Chinchwad'), (3, 'rented', 'Ahmednagar'), (4, 'rented', 'Akola'), (11, 'rented', 'Ambarnath'), (9, 'rented', 'Amravati'), (6, 'rented', 'Aurangabad'), (3, 'rented', 'Bhiwandi'), (7, 'rented', 'Bhusawal'), (1, 'rented', 'Chandrapur'), (11, 'rented', 'Dhule'), (10, 'rented', 'Ichalkaranji'), (5,

'There are 5 members from the state of Maharashtra with the following house ownership statuses and current cities they are living in:\n1. House Status: Own, Current City: Mumbai\n2. House Status: Rent, Current City: Pune\n3. House Status: Own, Current City: Nagpur\n4. House Status: Rent, Current City: Thane\n5. House Status: Own, Current City: Nashik'

In [12]:
generate("provide me state which is having high no. of applicants")



> Entering new SQLDatabaseChain chain...
provide me state which is having high no. of applicants
SQLQuery:SELECT `STATE`, COUNT(`Id`) AS `No_of_Applicants`
FROM `book_`
GROUP BY `STATE`
ORDER BY `No_of_Applicants` DESC
LIMIT 1;
SQLResult: [('Uttar_Pradesh', 279)]
Answer:Uttar Pradesh
> Finished chain.


'Uttar Pradesh is the state with the highest number of loan applicants in our database.'

In [13]:
generate(" Provide me no. of people whose martial status is single and having income worth more than 100000 and having current job status ")



> Entering new SQLDatabaseChain chain...
 Provide me no. of people whose martial status is single and having income worth more than 100000 and having current job status 
SQLQuery:SELECT COUNT(*) AS Num_People
FROM book_
WHERE Married = 'single' AND Income > 100000 AND CURRENT_JOB_YRS > 0;
SQLResult: [(2105,)]
Answer:There are 2105 people whose marital status is single, income is more than 100000, and have a current job status.
> Finished chain.


'There are 2105 people whose marital status is single, income is more than 100000, and have a current job status.'

In [14]:
generate("provide me names of applicants having profession as software engineer an having income between 1,00,000 to 1,50,000 and having martial status as single ")



> Entering new SQLDatabaseChain chain...
provide me names of applicants having profession as software engineer an having income between 1,00,000 to 1,50,000 and having martial status as single 
SQLQuery:SELECT `Id`, `Profession` 
FROM book_
WHERE `Profession` = 'Software_Developer' 
AND `Income` BETWEEN 100000 AND 150000 
AND `Married` = 'single'
LIMIT 5;
SQLResult: [(484, 'Software_Developer')]
Answer:The names of applicants having the profession as Software Developer, income between 100,000 and 150,000, and marital status as single are: Software_Developer.
> Finished chain.


'The names of applicants meeting your criteria are: Software_Developer.'

In [15]:
generate(" provide me the names of the state where applicants are having redflags and having house ownership as yes and having no car ownership ")



> Entering new SQLDatabaseChain chain...
 provide me the names of the state where applicants are having redflags and having house ownership as yes and having no car ownership 
SQLQuery:SELECT DISTINCT `STATE` 
FROM book_ 
WHERE `Risk_Flag` = 1 AND `House_Ownership` = 'owned' AND `Car_Ownership` = 'no'
LIMIT 5;
SQLResult: [('Jharkhand',), ('Haryana',), ('Uttar_Pradesh',), ('Punjab',), ('West_Bengal',)]
Answer:Jharkhand, Haryana, Uttar Pradesh, Punjab, West Bengal
> Finished chain.


'Applicants with red flags, house ownership as yes, and no car ownership are located in Haryana and Punjab.'

In [16]:
generate("provide me list of different professions applicants are having and how many members are in that profession")



> Entering new SQLDatabaseChain chain...
provide me list of different professions applicants are having and how many members are in that profession
SQLQuery:SELECT `Profession`, COUNT(`Profession`) AS `Number_of_Applicants` 
FROM book_ 
GROUP BY `Profession`
SQLResult: [('Air_traffic_controller', 55), ('Analyst', 42), ('Architect', 42), ('Army_officer', 43), ('Artist', 49), ('Aviator', 43), ('Biomedical_Engineer', 58), ('Chartered_Accountant', 44), ('Chef', 44), ('Chemical_engineer', 43), ('Civil_engineer', 38), ('Civil_servant', 43), ('Comedian', 59), ('Computer_hardware_engineer', 47), ('Computer_operator', 42), ('Consultant', 42), ('Dentist', 50), ('Designer', 48), ('Design_Engineer', 38), ('Drafter', 50), ('Economist', 51), ('Engineer', 49), ('Fashion_Designer', 59), ('Financial_Analyst', 53), ('Firefighter', 49), ('Flight_attendant', 59), ('Geologist', 43), ('Graphic_Designer', 57), ('Hotel_Manager', 51), ('Industrial_Engineer', 55), ('Lawyer', 53), ('Librarian', 44), ('Magistra

'Air_traffic_controller: 55\nAnalyst: 42\nArchitect: 42\nArmy_officer: 43\nArtist: 49\nAviator: 43\nBiomedical_Engineer: 58\nChartered_Accountant: 44\nChef: 44\nChemical_engineer: 43\nCivil_engineer: 38\nCivil_servant: 43\nComedian: 59\nComputer_hardware_engineer: 47\nComputer_operator: 42\nConsultant: 42\nDentist: 50\nDesigner: 48\nDesign_Engineer: 38\nDrafter: 50\nEconomist: 51\nEngineer: 49\nFashion_Designer: 59\nFinancial_Analyst: 53\nFirefighter: 49\nFlight_attendant: 59\nGeologist: 43\nGraphic_Designer: 57\nHotel_Manager: 51\nIndustrial_Engineer: 55\nLawyer: 53\nLibrarian: 44\nMagistrate: 62\nMechanical_engineer: 55\nMicrobiologist: 44\nOfficial: 34\nPetroleum_Engineer: 49\nPhysician: 70\nPolice_officer: 51\nPolitician: 57\nPsychologist: 55\nScientist: 43\nSecretary: 57\nSoftware_Developer: 52\nStatistician: 43\nSurgeon: 50\nSurveyor: 48\nTechnical_writer: 55\nTechnician: 37\nTechnology_specialist: 47\nWeb_designer: 47'

In [20]:
generate("provide me applicants who are from Telangana and having profession as flight attendents ")



> Entering new SQLDatabaseChain chain...
provide me applicants who are from Telangana and having profession as flight attendents 
SQLQuery:SELECT `Id`, `Income`, `Age`, `Experience`, `Married`, `House_Ownership`, `Car_Ownership`, `Profession`, `CITY`, `STATE`, `CURRENT_JOB_YRS`, `CURRENT_HOUSE_YRS`, `Risk_Flag`
FROM book_
WHERE `STATE` = 'Telangana' AND `Profession` = 'Flight_attendent'
LIMIT 5;
SQLResult: 
Answer:Answer: There are no applicants from Telangana with the profession of Flight Attendant in the database.
> Finished chain.


'There are no applicants from Telangana with the profession of Flight Attendant in the database.'

In [27]:
generate("provide me applicants who are from Telangana and having profession as comedian ")



> Entering new SQLDatabaseChain chain...
provide me applicants who are from Telangana and having profession as comedian 
SQLQuery:SELECT `Id`, `Income`, `Age`, `Experience`, `Married`, `House_Ownership`, `Car_Ownership`, `Profession`, `CITY`, `STATE`, `CURRENT_JOB_YRS`, `CURRENT_HOUSE_YRS`, `Risk_Flag` 
FROM book_
WHERE `STATE` = 'Telangana' AND `Profession` = 'comedian'
LIMIT 5;
SQLResult: [(510, 3177996, 28, 1, 'single', 'rented', 'no', 'Comedian', 'Nizamabad', 'Telangana', 1, 12, 0), (1436, 5288986, 47, 3, 'single', 'rented', 'no', 'Comedian', 'Miryalaguda', 'Telangana', 3, 12, 0), (1751, 4433480, 27, 9, 'single', 'rented', 'no', 'Comedian', 'Ramagundam[27]', 'Telangana', 9, 12, 0)]
Answer:There are 3 applicants from Telangana with the profession as a comedian.
> Finished chain.


'1. Applicant 1:\n   - Income: $30,000\n   - Age: 35\n   - Experience: 8 years\n   - Marital Status: Single\n   - House Ownership: Rent\n   - Car Ownership: No\n   - Profession: Comedian\n   - City: Hyderabad\n   - State: Telangana\n   - Current Job Status: Full-time\n   - Current Housing Status: Rent\n   - Risk Flag: Low\n   \n2. Applicant 2:\n   - Income: $25,000\n   - Age: 40\n   - Experience: 10 years\n   - Marital Status: Married\n   - House Ownership: Own\n   - Car Ownership: Yes\n   - Profession: Comedian\n   - City: Warangal\n   - State: Telangana\n   - Current Job Status: Part-time\n   - Current Housing Status: Own\n   - Risk Flag: Medium\n   \n3. Applicant 3:\n   - Income: $20,000\n   - Age: 28\n   - Experience: 5 years\n   - Marital Status: Single\n   - House Ownership: Rent\n   - Car Ownership: No\n   - Profession: Comedian\n   - City: Karimnagar\n   - State: Telangana\n   - Current Job Status: Full-time\n   - Current Housing Status: Rent\n   - Risk Flag: Low'

In [28]:
generate("provide me names of cities in which persons profession is police")



> Entering new SQLDatabaseChain chain...
provide me names of cities in which persons profession is police
SQLQuery:SELECT DISTINCT `CITY` 
FROM book_ 
WHERE `Profession` = 'Police'
LIMIT 5;
SQLResult: 
Answer:Answer: There are no cities in the database where a person's profession is listed as "Police".
> Finished chain.


'There are no cities in the database where a person\'s profession is listed as "Police".'